# **Celestial Object Tracker**
### Converting Right Ascension - Declination (RA - DEC) to Altitude - Azimuth (ALT - AZ) Coordinates

>**Some calculation can be done easily using libraries like `astropy`. However, since the final programming language and approach is still unknown, this approach of coding is used. Furthermore, it is beneficial to familiarize with the algorithms used.**

### Function Definitions

First necessary modules and functions are imported. Then some required functions are defined.

`date` and `time` functions return a tuple. These values can be simply passed as arguments, in this case. However, since the final programming languange is not known yet, this approach is used.

In [511]:
from datetime import datetime
from math import pi, radians, degrees, sin, cos, tan, asin, acos, atan2

In [512]:
def date(dd, mm, yyyy):
  return (dd, mm, yyyy)

In [513]:
def time(h, min, sec):
  return (h, min, sec)

Many functions require times in hours, and angles in degrees. Nevertheless, many resources provide values in hours, minutes, seconds or degrees, arcminutes, arcseconds format. The following function `convert_60_60` is used to convert those values.

In [514]:
def convert_60_60(hours, minutes, seconds, negative=False):
  magnitude = float(hours + (minutes / 60) + (seconds / 3600))
  signed_magnitude = (-1) ** negative * magnitude
  return signed_magnitude

**Julian date** is defined as the number of days elapsed since the 1st of January 4713 BC. The following function `julian_date' calculated the julian date of a given date considering the facts such as leap years, and the switching to the Gregorian calendar in 1582.

In [515]:
# def julian_date(date):
#   d, m, y = date

#   years = 4713 + y - 1 # + 1
#   leap_years_1 = (4713 // 4) + (1) + (1582 // 4)
#   leap_years_2 = (y - 1 - 1579) // 4 - (y - 1 - 1500) // 100 + (y - 1 - 1600) // 400
#   leap_years = leap_years_1 + leap_years_2

#   days_1 = (leap_years * 366) + ((years - leap_years) * 365) - 12 # + 2
#   m_days = [0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334, 365]
#   days_2 = m_days[m - 1] + d
#   if (((y % 4 == 0) and (y % 100 != 0)) or (y % 400 == 0)) and m > 2:
#     days_2 += 1
#   days = days_1 + days_2

#   return days # days

However, since we are mainly interested in the contemporary time period, there is no need for tedious calculations since 4713 BC. Instead, we can implement the following algorithm to easily calculate the Julian date from 2000 CE - 2100 CE. (It might work beyond that range as well, but not guranteed.)

In [516]:
# This is a simplified version. This will perfectly work in range 2000 - 2100
def julian_date(date):
  d, m, y = date

  years = y - 2000
  leap_years = 0
  for year in range(2000, y):
    if year % 4 == 0: leap_years += 1

  days_1 = (leap_years * 366) + ((years - leap_years) * 365)

  m_days = [0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334, 365]
  m_days_leap = [0, 31, 60, 91, 121, 152, 182, 213, 244, 274, 305, 335, 366]

  if y % 4 == 0:
    days_2 = m_days_leap[m - 1]
  else:
    days_2 = m_days[m - 1]

  days = days_1 + days_2 + (d - 1)

  return days # days

**Juilan time** is similarly the time elapsed since the reference date in Julian date. Hence, some fraction of a day is added to the Julian date to calculate the Julian time.

In [518]:
def julian_time(date, time):
  h, min, sec = time

  jdate = julian_date(date)

  hours = (h) + (min / 60) + (sec / 3600)
  if h > 12:
    hours -= 12

  jtime = jdate + (hours / 24)
  return jtime  # days

>Note that the units of both Julian date and Julian time is **days**.

Here, let's calculate the Julian date and Julian time of the current moment.

> Note that they are defined for the date and time at Greenwich.

> We later consider our local time and location for further calculations.

In [519]:
now_ = datetime.utcnow()
print(now_)
year, month, day = now_.year, now_.month, now_.day
hour, minute, second = now_.hour, now_.minute, now_.second

date1 = date(day, month, year)
time1 = time(hour, minute, second)

print(julian_date(date1))
print(julian_time(date1, time1))

2024-01-08 17:06:01.165355
8773
8773.212511574075


**Sidereal time** is the time used in astronomical purposes. It differs from what we use normally by only a small amount in short run. This closely corresponds to the precision of Earth's axis (period: approx. 26,000 years) and the Earth having a day of 23h 56min 04sec, not exactly 24 hours.

**GMST** stands for Greenwich Mean Sidereal Time. Here with this function `gmst` we calculate the GMST. (i.e., sidereal time at Greenwich)

In [520]:
def gmst(date, time):
  d_tt = julian_time(date, time) # - 2451545.0
  t = d_tt / 36525.0
  gmst_ = 280.46061837 + (360.98564736629 * d_tt) + (0.000387933 * (t ** 2)) - ((t ** 3) / 38710000.0)
  gmst_ = gmst_ % 360 # degrees
  gmst_ = gmst_ / (360 / 24)  # hours
  return gmst_

In [521]:
print(gmst(date1, time1))

0.2839075569063425


Here, now we start considering our local position on the Earth. We can use the GPS coordinates for specifying our location as latitude and longitude.

> Note that North latitudes and East longitudes have positive values, while South latitudes and West longitudes have negative values.

In [522]:
def location(latitude, longitude):
  return (latitude, longitude)

**λ_h** is a measure of the longitudal difference between the current location of the observer and Greenwich. This is measured in hours.


In [523]:
def lambda_h(location):
  return float(location[1] / 360 * 24) # hours

**LMST** stand for Local Mean Sidereal Time. Here, now we consider our actual location on Earth, and calculate the sidereal time corresponds to that location.

LMST = GMST + λ_h

In [524]:
def lmst(date, time, location):
  gmst_ = gmst(date, time)
  l_h = lambda_h(location)
  lmst_ = (gmst_ + l_h) % 24
  return lmst_ # hours

Let's do some examples. Here we calculate the local mean sidereal times of Colombo and Kurunegala. We need the GPS coordinates of the location. (Actually, only the longitude value is needed to calculate LMST.)

In [525]:
location_colombo = location(6.9271, 79.8612)
print(location_colombo)
print(lmst(date1, time1, location_colombo))

(6.9271, 79.8612)
5.607987556906342


In [526]:
location_kurunegala = location(7.4818, 80.3609)
print(location_kurunegala)
print(lmst(date1, time1, location_kurunegala))

(7.4818, 80.3609)
5.641300890239676


>**The results from the code up to this point match to the results in the Internet. Hence, it is highly likely that the code up to this point is correct.**

RA - DEC coordinated have two values, namely the RA (Right Ascension) and DEC (Declination) values.

In [527]:
def ra_dec_coordinate(ra, dec):
  return (ra, dec)  # hours, degrees

Let's do an example for the brightest star in the night sky, Sirius.

> Note that we have to find the RA and DEC values from the internet or any other resource.

In [528]:
sirius = ra_dec_coordinate(convert_60_60(6, 45, 9), convert_60_60(16, 48, 52, negative=True))
print(sirius)

(6.7525, -16.814444444444444)


**Hour angle** measures the difference in the right ascension time of any stellar object depending on the current location.

In [529]:
def hour_angle(date, time, location, ra_dec):
  lst = lmst(date, time, location)  # hours
  ra = ra_dec[0]  # hours
  ha_ = lst - ra

  if ha_ < 0: ha_ = ha_ + 24

  return ha_  # hours

>**For a keen observer, it might be visible that these concepts can be simplified further, because *λ_h* and *hour angle* kind of do the same thing.Since we are adding and subtracting those values, one might be able to simplify the algorithm, at the cost of loosing the readbility.**

***The following is the most important function.***

The function `radec_to_altaz` converts the RA - DEC coordinates to local ALT - AZ coordinates.

> It should be noted that several resources give slighlty different algorithms and the calculations done using the currently implemented algorithm might not be the best or even correct for some objects.

In [530]:
# def radec_to_altaz(date, time, location, ra_dec):
#   ha = hour_angle(date, time, location, ra_dec) # hours
#   ha = ha * (2 * pi / 24) # radians
#   lat = location[0] # degrees
#   lat = radians(lat)  # radians

#   ra, dec = ra_dec  # hours, hours
#   # dec = pi - dec
#   ra, dec = ra * (2 * pi / 24), dec * (2 * pi / 24) # radians, radians

#   altitude = asin(sin(dec) * sin(lat) + cos(dec) * cos(lat) * cos(ha))  # radians
#   azimuth = asin(-1 * sin(ha) * cos(dec) / cos(altitude)) # radians

#   # altitude = max(0, altitude)
#   azimuth = azimuth if ha >= 12 else 2 * pi - azimuth

#   return (degrees(altitude), degrees(azimuth))

In [531]:
def radec_to_altaz(date, time, location, ra_dec):
  ha = hour_angle(date, time, location, ra_dec) # hours
  ha = ha * (2 * pi / 24) # radians

  lat = location[0] # degrees
  lat = radians(lat)  # radians

  ra, dec = ra_dec  # hours, degrees
  ra, dec = ra * (2 * pi / 24), radians(dec) # radians, radians

  altitude = asin(sin(dec) * sin(lat) + cos(dec) * cos(lat) * cos(ha))  # radians
  azimuth = acos((sin(dec) - sin(altitude) * sin(lat)) / (cos(altitude) * cos(lat)))  # radians

  if sin(ha) >= 0:
    azimuth = 2 * pi - azimuth

  return (degrees(altitude), degrees(azimuth))

### Testing the Algorithm

Let's see what is the ALT - AZ coordinate of Sirius. This value depends on the current location and time.

In [532]:
sirius_altaz = radec_to_altaz(date1, time1, location_kurunegala, sirius)
print(sirius_altaz)

(60.639490546986096, 145.94522122021576)


Some more examples to test the algorithm.

In [533]:
the_sun = ra_dec_coordinate(convert_60_60(19, 17, 1), convert_60_60(22, 15, 31, negative=True))
print(the_sun)
the_sun_altaz = radec_to_altaz(date1, time1, location_kurunegala, the_sun)
print(the_sun_altaz)

(19.283611111111114, -22.25861111111111)
(-62.05650798090731, 235.4102883127599)


In [534]:
jupiter = ra_dec_coordinate(convert_60_60(2, 13, 50), convert_60_60(12, 13, 30))
print(jupiter)
jupiter_altaz = radec_to_altaz(date1, time1, location_kurunegala, jupiter)
print(jupiter_altaz)

(2.2305555555555556, 12.225)
(39.43963041940236, 279.7012950941959)


In [535]:
deneb = ra_dec_coordinate(convert_60_60(20, 41, 26), convert_60_60(45, 16, 49))
print(deneb)
deneb_altaz = radec_to_altaz(date1, time1, location_kurunegala, deneb)
print(deneb_altaz)

(20.690555555555555, 45.280277777777776)
(-23.227784290747582, 326.744667491315)


In [536]:
polaris = ra_dec_coordinate(convert_60_60(2, 31, 49), convert_60_60(89, 15, 51))
print(polaris)
polaris_altaz = radec_to_altaz(date1, time1, location_kurunegala, polaris)
print(polaris_altaz)

(2.5302777777777776, 89.26416666666667)
(7.986435768194149, 359.4595508154153)


In [537]:
alpheratz = ra_dec_coordinate(convert_60_60(0, 8, 23), convert_60_60(29, 5, 26))
print(alpheratz)
alpheratz_altaz = radec_to_altaz(date1, time1, location_kurunegala, alpheratz)
print(alpheratz_altaz)

(0.13972222222222222, 29.090555555555554)
(10.13930740739531, 298.33719046432105)


>**The results from the code match to the results in the Internet. Hence, it is highly likely that the code is correct.**